<a href="https://colab.research.google.com/github/apschlissel/w266-final-project/blob/main/BERT_reddit_baseline_(Kevin).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install -q transformers

In [44]:
!pip install simpletransformers

In [45]:
# Useful Links / Source Code
# Source: https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c
# Source: https://medium.com/@senthilnathangautham/colab-gcp-compute-how-to-link-them-together-98747e8d940e
# Source: https://stackoverflow.com/questions/49031798/when-i-use-google-colaboratory-how-to-save-image-weights-in-my-google-drive

In [46]:
from __future__ import print_function
import ipywidgets as widgets
from transformers import pipeline
from simpletransformers.t5 import T5Model, T5Args
import pandas as pd
import logging
import numpy as np
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import json
# Pull reddit data from reddit api
import requests

In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [48]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('pigKA_TKnDkXcatEGcbo8g', 'nawGKK2MfPtC6vKz8TjaNEnmYfAggA')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'Katsuuu100',
        'password': 'Testing159753'}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [58]:
# Source: https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

# Pull from 5 classes. 5 classes = 5 subreddits.

# url_list = ["https://oauth.reddit.com/r/wallstreetbets/new",
            # "https://oauth.reddit.com/r/teenagers/new",
            # "https://oauth.reddit.com/r/copypasta/new",
              # "https://oauth.reddit.com/r/genz/new",
              # "https://oauth.reddit.com/r/unpopularopinion/new"]

my_list_of_dictionaries = []
total = 0

for i in range(5):
  page_count = 25
  for j in range(10):
    url_list = [f"https://oauth.reddit.com/r/wallstreetbets/new/?count={page_count}&after=t3_tftdpj",
            f"https://oauth.reddit.com/r/teenagers/new/?count={page_count}&after=t3_tftdpj",
            f"https://oauth.reddit.com/r/copypasta/new/?count={page_count}&after=t3_tftdpj",
            f"https://oauth.reddit.com/r/genz/new/?count={page_count}&after=t3_tftdpj",
            f"https://oauth.reddit.com/r/unpopularopinion/new/?count={page_count}&after=t3_tftdpj",
            ]
    print(url_list[i])
    res = requests.get(url_list[i],
                    headers=headers)
                    # params={"limit": "100"})

    #print(json.dumps(res.json(), indent=4, sort_keys=False))

    reddit_dictionary = res.json()
    # print(json.dumps(reddit_dictionary["data"]["children"][0], indent=4, sort_keys=False))
    # print(len(reddit_dictionary["data"]["children"]))
    for k in range(len(reddit_dictionary["data"]["children"])):
      my_dictionary = {}
      my_dictionary["subreddit"] = reddit_dictionary["data"]["children"][k]["data"]["subreddit"]
      my_dictionary["text"] = reddit_dictionary["data"]["children"][k]["data"]["selftext"]
      my_dictionary["title"] = reddit_dictionary["data"]["children"][k]["data"]["title"]
      my_dictionary["url"] = reddit_dictionary["data"]["children"][k]["data"]["url"]
      # print(reddit_dictionary["data"]["children"][k]["data"]["subreddit"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["selftext"])
      # print(reddit_dictionary["data"]["children"][k]["data"]["url"])
      my_list_of_dictionaries.append(my_dictionary)
      total += 1
    print(json.dumps(my_list_of_dictionaries, indent=4, sort_keys=False))
    page_count += 25
    total += 25
print(total)

Streaming output truncated to the last 5000 lines.
        "title": "The dude whose portfolio was funded by gram gram and went from $200k to $9k.",
        "url": "https://v.redd.it/7xrcf66ensn81"
    },
    {
        "subreddit": "wallstreetbets",
        "text": "Major international trading firm gets a margin call.  On the lookout for a cash infusion.\n\nTrade accordingly.\n\n[https://www.reuters.com/business/trafigura-seeks-funding-commodity-surge-triggers-margin-calls-bloomberg-news-2022-03-15/](https://www.reuters.com/business/trafigura-seeks-funding-commodity-surge-triggers-margin-calls-bloomberg-news-2022-03-15/)",
        "title": "Marge called. Pick up the phone, please, someone...",
        "url": "https://www.reddit.com/r/wallstreetbets/comments/tfpohf/marge_called_pick_up_the_phone_please_someone/"
    },
    {
        "subreddit": "wallstreetbets",
        "text": "",
        "title": "Turned 83\u00a2 \u2014&gt; $1440 in one day. The 1440 will be gone by end of next week."

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
        "title": "The dude whose portfolio was funded by gram gram and went from $200k to $9k.",
        "url": "https://v.redd.it/7xrcf66ensn81"
    },
    {
        "subreddit": "wallstreetbets",
        "text": "Major international trading firm gets a margin call.  On the lookout for a cash infusion.\n\nTrade accordingly.\n\n[https://www.reuters.com/business/trafigura-seeks-funding-commodity-surge-triggers-margin-calls-bloomberg-news-2022-03-15/](https://www.reuters.com/business/trafigura-seeks-funding-commodity-surge-triggers-margin-calls-bloomberg-news-2022-03-15/)",
        "title": "Marge called. Pick up the phone, please, someone...",
        "url": "https://www.reddit.com/r/wallstreetbets/comments/tfpohf/marge_called_pick_up_the_phone_please_someone/"
    },
    {
        "subreddit": "wallstreetbets",
        "text": "",
        "title": "Turned 83\u00a2 \u2014&gt; $1440 in one day. The 1440 will be gone by end of next week."

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
        "title": "The dude whose portfolio was funded by gram gram and went from $200k to $9k.",
        "url": "https://v.redd.it/7xrcf66ensn81"
    },
    {
        "subreddit": "wallstreetbets",
        "text": "Major international trading firm gets a margin call.  On the lookout for a cash infusion.\n\nTrade accordingly.\n\n[https://www.reuters.com/business/trafigura-seeks-funding-commodity-surge-triggers-margin-calls-bloomberg-news-2022-03-15/](https://www.reuters.com/business/trafigura-seeks-funding-commodity-surge-triggers-margin-calls-bloomberg-news-2022-03-15/)",
        "title": "Marge called. Pick up the phone, please, someone...",
        "url": "https://www.reddit.com/r/wallstreetbets/comments/tfpohf/marge_called_pick_up_the_phone_please_someone/"
    },
    {
        "subreddit": "wallstreetbets",
        "text": "",
        "title": "Turned 83\u00a2 \u2014&gt; $1440 in one day. The 1440 will be gone by end of next week."

In [59]:
# Source: https://github.com/susanli2016/NLP-with-Python/blob/master/Text_Classification_With_BERT.ipynb
# Convert list of dictionaries into pandas df
df = pd.DataFrame(my_list_of_dictionaries)
df.head()

,subreddit,text,title,url
0,wallstreetbets,,When You Yolo All Your Money In China Tech Bec...,https://i.redd.it/v16sa4n57tn81.jpg
1,wallstreetbets,,First time trading with options. Why did I onl...,https://i.redd.it/yp9t52il6tn81.jpg
2,wallstreetbets,When Volker raised interest rates to 20% in 19...,How the fuck can JPo raise rates? He cannot! B...,https://www.reddit.com/r/wallstreetbets/commen...
3,wallstreetbets,,BABA to the MOON 🚀 🚀 🚀,https://i.redd.it/o3wf2iev1tn81.jpg
4,wallstreetbets,,A training video for anyone heading back to We...,https://v.redd.it/cxtare7gzsn81


In [60]:
df['subreddit'].value_counts()

wallstreetbets    250
Name: subreddit, dtype: int64

In [61]:
possible_labels = df.subreddit.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'wallstreetbets': 0}

In [62]:
df['label'] = df.subreddit.replace(label_dict)

In [63]:
df.head()

,subreddit,text,title,url,label
0,wallstreetbets,,When You Yolo All Your Money In China Tech Bec...,https://i.redd.it/v16sa4n57tn81.jpg,0
1,wallstreetbets,,First time trading with options. Why did I onl...,https://i.redd.it/yp9t52il6tn81.jpg,0
2,wallstreetbets,When Volker raised interest rates to 20% in 19...,How the fuck can JPo raise rates? He cannot! B...,https://www.reddit.com/r/wallstreetbets/commen...,0
3,wallstreetbets,,BABA to the MOON 🚀 🚀 🚀,https://i.redd.it/o3wf2iev1tn81.jpg,0
4,wallstreetbets,,A training video for anyone heading back to We...,https://v.redd.it/cxtare7gzsn81,0


In [64]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [65]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [66]:
df.groupby(['subreddit', 'label', 'data_type']).count()

text  title  url
subreddit      label data_type                  
wallstreetbets 0     train       212    212  212
                     val          38     38   38

In [67]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [68]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [69]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [70]:
len(dataset_train), len(dataset_val)

(212, 38)

In [71]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [72]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [73]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [74]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [75]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [76]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [77]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [78]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [79]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/gdrive/MyDrive/W266/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/71 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/W266/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)